In [1]:
from __future__ import absolute_import
from mimseq import version
from mimseq.tRNAtools import TRNA
#from mimseq.tRNAtools import modsToSNPIndex, generateGSNAPIndices, newModsParser, tidyFiles
#from mimseq.tRNAmap import mainAlign
#from mimseq.getCoverage import getCoverage, plotCoverage
#from mimseq.mmQuant import generateModsTable, plotCCA
#from mimseq.ssAlign import structureParser, modContext 
#from mimseq.splitClusters import splitIsodecoder, unsplitClusters, getIsodecoderSizes, writeIsodecoderTranscripts
import sys, os, subprocess, logging, datetime, copy, shutil, re, glob
import argparse
from pyfiglet import figlet_format
from collections import defaultdict

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML
from pathlib import Path
import pandas as pd
import requests
from requests.models import HTTPError




log = logging.getLogger(__name__)

%load_ext autoreload
%autoreload 2

In [2]:
def restrictedFloat(x):
## Method for restricting cluster_id and cov_diff argument to float between 0 and 1
    try:
        x = float(x)
        if x < 0.0 or x > 1.0:
            raise argparse.ArgumentTypeError('{} not in range 0.0 - 1.0'.format(x))
        return x
    except ValueError:
        raise argparse.ArgumentTypeError('{} not a real number'.format(x))

def restrictedFloat2(x):
## Method for restricting min-cov argument to float between 0 and 1, or int greater than 1

    try:
        x = float(x)
        if x < 0.0:
            raise argparse.ArgumentTypeError('{} not greater than 0'.format(x))
        return x
    except ValueError:
        raise argparse.ArgumentTypeError('{} not a real number'.format(x))



parser = argparse.ArgumentParser(description = 'Custom high-throughput tRNA sequencing alignment and quantification pipeline\
    based on modification induced misincorporation cDNA synthesis.', add_help = True, usage = "%(prog)s [options] sample data")

inputs = parser.add_argument_group("Input files")
inputs.add_argument('-s','--species', metavar='species', required = not ('-t' in sys.argv), dest = 'species', help = \
    'Species being analyzed for which to load pre-packaged data files (prioritized over -t, -o and -m). Options are: Hsap, Hsap38, Mmus, Scer, Spom, Dmel, Drer, Ecol', \
    choices = ['Hsap','Hsap38','Mmus','Scer','Spom','Dmel', 'Drer', 'Ecol'])
inputs.add_argument('-t', '--trnas', metavar='genomic tRNAs', required = False, dest = 'trnas', help = \
    'Genomic tRNA fasta file, e.g. from gtRNAdb or tRNAscan-SE. Already avalable in data folder for a few model organisms.')
inputs.add_argument('-o', '--trnaout', metavar = 'tRNA out file', required = (not '--species' or '-s' in sys.argv) or ('-t' in sys.argv), 
    dest = 'trnaout', help = 'tRNA.out file generated by tRNAscan-SE (also may be available on gtRNAdb). Contains information about tRNA features, including introns.')
inputs.add_argument('-m', '--mito-trnas', metavar = 'mitochondrial tRNAs', required = False, dest = 'mito', \
    help = 'Mitochondrial tRNA fasta file. Should be downloaded from mitotRNAdb for species of interest. Already available in data folder for a few model organisms.')

options = parser.add_argument_group("Program options")
options.add_argument('--pretRNAs', required = False, dest = 'pretrnas', action = 'store_true',\
    help = "Input reference sequences are pretRNAs. Enabling this option will disable the removal of intron sequences and addition of 3'-CCA to generate \
    mature tRNA sequences. Useful for mapping and discovering pretRNA sequence reads.")
options.add_argument('--no-cluster', required = False, dest = 'cluster', action = 'store_false',\
    help = 'Disable usearch sequence clustering of tRNAs by isodecoder which drastically reduces the rate of multi-mapping reads. Default is enabled.')
options.add_argument('--cluster-id', metavar = 'clustering identity threshold', dest = 'cluster_id', type = restrictedFloat, nargs = '?', default = 0.97,\
    required = False, help = 'Identity cutoff for usearch clustering between 0 and 1. Default is 0.97.')
options.add_argument('--deconv-cov-ratio', metavar='deconvolution coverage threshold', dest='cov_diff', type = restrictedFloat, nargs = '?', default=0.5,\
    required=False, help="Threshold for ratio between coverage at 3' end and mismatch used for deconvolution. Coverage reductions greater than the threshold will result in non-deconvoluted sequences. \
        Default is 0.5 (i.e. less than 50%% reduction required for deconvolution).")
options.add_argument('--threads', metavar = 'thread number', required = False, dest = 'threads', type = int, \
    help = 'Set processor threads to use during read alignment and read counting.')
options.add_argument('--posttrans-mod-off', required = False, dest = 'posttrans', action = 'store_true', \
    help = "Disable post-transcriptional modification of tRNAs, i.e. addition of 3'-CCA and 5'-G (His) to mature sequences. Disable for certain \
    prokaryotes (e.g. E. coli) where this is genomically encoded. Leave enabled (default) for all eukaryotes.")
options.add_argument('--control-condition', metavar = 'control condition', required = True, dest = 'control_cond', \
    help = 'Name of control/wild-type condition as per user defined group specified in sample data input. This must exactly match the group name \
    specified in sample data. This is used for differential expression analysis so that results are always in the form mutant/treatment vs WT/control. REQUIRED')
options.add_argument('--no-cca-analysis', required = False, dest = 'cca', action = 'store_false',\
    help = "Disable analysis of 3'-CCA ends. When enabled, this calculates proportions of CC vs CCA ending reads per cluster and performs DESeq2 analysis. \
    Useful for comparing functional to non-functional mature tRNAs. Default is enabled.")
options.add_argument('--double-cca', required = False, dest = 'double_cca', action = "store_true",\
    help = "Enable analysis of 3'-CCACCA tagging for tRNA degradation pathway. Note that this will alter the output of the CCA analysis pipeline.")
options.add_argument('--local-modomics', required=False, dest = 'local_mod', action='store_true',\
    help = "Disable retrieval of Modomics data from online. Instead use older locally stored data. Warning - this leads\
        to usage of older Modomics data!")

align = parser.add_argument_group("GSNAP alignment options")
align.add_argument('--max-mismatches', metavar = 'allowed mismatches', required = False, dest = 'mismatches', type = float, \
    help = 'Maximum mismatches allowed. If specified between 0.0 and 1.0, then treated as a fraction of read length. Otherwise, treated as \
    integer number of mismatches. Default is an automatic ultrafast value calculated by GSNAP; see GSNAP help for more info.')
align.add_argument('--remap-mismatches', metavar = 'allowed mismatches for remap', required = False, dest = 'remap_mismatches', type = float,\
    help = 'Maximum number of mismatches allowed during remapping of all reads. Treated similarly to --max-mismatches. This is important to control misalignment of reads to similar clusters/tRNAs \
    Note that the SNP index will be updated with new SNPs from the first round of alignment and so this should be relatively small to prohibit misalignment.')
align.add_argument('--no-snp-tolerance', required = False, dest = 'snp_tolerance', action = 'store_false',\
    help = 'Disable GSNAP SNP-tolerant read alignment, where known modifications from Modomics are mapped as SNPs. Default is enabled.')


outputs = parser.add_argument_group("Output options")
outputs.add_argument('-n', '--name', metavar = 'experiment name', required = True, dest = 'name', help = \
    'Name of experiment. Note, output files and indices will have this as a prefix. REQUIRED')
outputs.add_argument('--out-dir', metavar = 'output directory', default="./", required = False, dest = 'outdir', help = \
    'Output directory. Default is current directory. Cannot be an existing directory.')
outputs.add_argument('--keep-temp', required = False, dest='keep_temp', action = 'store_true', help = \
    'Keeps multi-mapping and unmapped bam files from GSNAP alignments. Default is false.')
outputs.add_argument('--overwrite', required = False, dest='overwrite', action = 'store_true', help = \
    'Overwrite existing output directory if overlaps with --our-dir.')

bedtools = parser.add_argument_group("Bedtools coverage options")
bedtools.add_argument('--min-cov', metavar = 'Minimum coverage per cluster', required = False, dest = 'min_cov', type = restrictedFloat2, default=0.0005, \
    help = "Minimum coverage per cluster required to include this cluster in coverage plots, modification analysis, and 3'-CCA analysis. \
    Can be a fraction of total mapped reads between 0 and 1, or an integer of absolute coverage. Any cluster not meeting the threshold in 1 or more sample will be excluded. \
    Note that all clusters are included for differential expression analysis with DESeq2. Default = 0.0005 (0.05%% mapped reads).")
bedtools.add_argument('--max-multi', metavar = 'Bedtools coverage multithreading', required = False, dest = 'max_multi', type = int, \
    help = 'Maximum number of bam files to run bedtools coverage on simultaneously. Increasing this number reduces processing time\
    by increasing number of files processed simultaneously. However, depending on the size of the bam files to process and\
    available memory, too many files processed at once can cause termination of mim-tRNAseq due to insufficient memory. If\
    mim-tRNAseq fails during coverage calculation, lower this number. Increase at your own discretion. Default is 3.')

remapping = parser.add_argument_group("Analysis of unannotated modifications and realignment")
remapping.add_argument('--remap', required = False, dest = 'remap', action = 'store_true',\
    help = 'Enable detection of unannotated (potential) modifications from misincorporation data. These are defined as having a total misincorporation rate\
    higher than the threshold set with --misinc-thresh. These modifications are then appended to already known ones, and read alignment is reperformed.\
    Very useful for poorly annotated species in Modomics. Due to realignment and misincorporation parsing, enabling this option slows the analysis down considerably.')
remapping.add_argument('--misinc-thresh', metavar = 'threshold for unannotated mods', dest = 'misinc_thresh', type = restrictedFloat, nargs = '?', default = 0.1,\
    required = False, help = 'Threshold of total misincorporation rate at a position in a cluster used to call unannotated modifications. Value between 0 and 1, default is 0.1  (10%% misincorporation).')

parser.add_argument('--version', action='version', version='%(prog)s {}'.format(version.__version__), help = 'Show version number and exit')
parser.add_argument('sampledata', help = 'Sample data sheet in text format, tab-separated. Column 1: full path to fastq (or fastq.gz). Column 2: condition/group.')

parser.set_defaults(threads=1, out="./", max_multi = 3, min_cov = 0, mito = '', cov_diff = 0.5)

In [3]:
args = parser.parse_args('--species Hsap --cluster-id 0.95 --threads 1 --min-cov 2000 --max-mismatches 0.1 --control-condition HEK293T -n hg19_test --out-dir hg19_HEK239vsK562_test --overwrite --max-multi 1 --remap --remap-mismatches 0.075 sampleData_HEKvsK562.txt'.split())


try:
    args.repo_path = os.path.dirname(os.path.realpath(__file__))
except NameError:
    filepath = '/Users/krdav/Google Drive/MCB/Sullivan_lab/tRNA_charging/mim-tRNAseq/mimseq/mimseq.py'
    args.repo_path = os.path.dirname(os.path.realpath(filepath))

In [4]:
print(figlet_format('mim-tRNAseq', font='standard'))
print(" Modification-induced misincorporation analysis of tRNA sequencing data\n")
if args.pretrnas:
    if args.cca:
        log.warning("Disabling CCA analysis in pre-tRNA mode...")
        args.cca = False
    if args.cluster:
        log.warning("Disabling tRNA clustering in pre-tRNA mode...")
        args.cluster = False
# Check that control_cond exists in sample data
conditions = list()
with open(args.sampledata, "r") as sampleData:
    for line in sampleData:
        line = line.strip()
        if not line.startswith("#"):
            conditions.append(line.split("\t")[1])
if args.control_cond not in conditions:
    raise argparse.ArgumentTypeError('{} not a valid condition in {}'.format(args.control_cond, args.sampledata))
if not args.species and not (args.trnas or args.trnaout):
    parser.error('Must specify valid --species argument or supply -t (tRNA sequences) and -o (tRNAscan out file)!')                        
else:
    if args.species:
        if args.species == 'Hsap':
            args.trnas = args.repo_path + "/data/hg19-eColitK/hg19_eColitK.fa"
            args.trnaout = args.repo_path + "/data/hg19-eColitK/hg19_eschColi-tRNAs.out"
            args.mito = args.repo_path + "/data/hg19-eColitK/hg19-mitotRNAs.fa"
        if args.species == 'Hsap38':
            args.trnas = args.repo_path + "/data/hg38-eColitK/hg38-tRNAs-filtered.fa"
            args.trnaout = args.repo_path + "/data/hg38-eColitK/hg38-tRNAs-detailed.out"
            args.mito = args.repo_path + "/data/hg38-eColitK/hg38-mitotRNAs.fa"
        if args.species == 'Scer':
            args.trnas = args.repo_path + "/data/sacCer3-eColitK/sacCer3_eschColitK.fa"
            args.trnaout = args.repo_path + "/data/sacCer3-eColitK/sacCer3_eschColi-tRNAs.out"
            args.mito = args.repo_path + "/data/sacCer3-eColitK/sacCer3-mitotRNAs.fa"
        if args.species == 'Mmus':
            args.trnas = args.repo_path + "/data/mm10-eColitK/mm10_eColitK-tRNAs.fa"
            args.trnaout = args.repo_path + "/data/mm10-eColitK/mm10_eschColi-tRNAs.out"
            args.mito = args.repo_path + "/data/mm10-eColitK/mm10-mitotRNAs.fa"
        if args.species == 'Spom':
            args.trnas = args.repo_path + "/data/schiPomb-eColitK/schiPomb_972H-tRNAs.fa"
            args.trnaout = args.repo_path + "/data/schiPomb-eColitK/schiPomb_eschColi-tRNAs.out"
            args.mito = args.repo_path + "/data/schiPomb-eColitK/schiPomb-mitotRNAs.fa"
        if args.species == 'Dmel':
            args.trnas = args.repo_path + "/data/dm6-eColitK/dm6_eColitK-tRNAs.fa"
            args.trnaout = args.repo_path + "/data/dm6-eColitK/dm6_eschColi-tRNAs.out"
            args.mito = args.repo_path + "/data/dm6-eColitK/dm6-mitotRNAs.fa"
        if args.species == 'Drer':
            args.trnas = args.repo_path + "/data/danRer11-eColitK/danRer11_eColitK_filtered.fa"
            args.trnaout = args.repo_path + "/data/danRer11-eColitK/danRer11_eschColi-tRNAs.out"
            args.mito = args.repo_path + "/data/danRer11-eColitK/danRer11-mitotRNAs.fa"
        if args.species == 'Ecol':
            args.trnas = args.repo_path + "/data/eschColi-K_12_MG1655-tRNAs/eschColi_K_12_MG1655-tRNAs.fa"
            args.trnaout = args.repo_path + "/data/eschColi-K_12_MG1655-tRNAs/eschColi_K_12_MG1655-tRNAs.out"
            args.mito = ''
    else:
        args.species = args.trnas.split("/")[-1].split(".")[0]

           _                 _   ____  _   _    _                   
 _ __ ___ (_)_ __ ___       | |_|  _ \| \ | |  / \   ___  ___  __ _ 
| '_ ` _ \| | '_ ` _ \ _____| __| |_) |  \| | / _ \ / __|/ _ \/ _` |
| | | | | | | | | | | |_____| |_|  _ <| |\  |/ ___ \\__ \  __/ (_| |
|_| |_| |_|_|_| |_| |_|      \__|_| \_\_| \_/_/   \_\___/\___|\__, |
                                                                 |_|

 Modification-induced misincorporation analysis of tRNA sequencing data



In [5]:
# trnas, trnaout, name, species, out, cluster, cluster_id, cov_diff, posttrans, control_cond, threads, max_multi, snp_tolerance, keep_temp, cca, double_cca, min_cov, mismatches, remap, remap_mismatches, misinc_thresh, mito_trnas, pretrnas, local_mod, sample_data = args.trnas, args.trnaout, args.name, args.species, args.out, args.cluster, args.cluster_id, args.cov_diff, args.posttrans, args.control_cond, args.threads, args.max_multi, args.snp_tolerance, args.keep_temp, args.cca, args.double_cca, args.min_cov, args.mismatches, args.remap, args.remap_mismatches, args.misinc_thresh, args.mito, args.pretrnas, args.local_mod, args.sampledata

In [6]:
args.local_mod = True
args

Namespace(cca=True, cluster=True, cluster_id=0.95, control_cond='HEK293T', cov_diff=0.5, double_cca=False, keep_temp=False, local_mod=True, max_multi=1, min_cov=2000.0, misinc_thresh=0.1, mismatches=0.1, mito='/Users/krdav/Google Drive/MCB/Sullivan_lab/tRNA_charging/mim-tRNAseq/mimseq/data/hg19-eColitK/hg19-mitotRNAs.fa', name='hg19_test', out='./', outdir='hg19_HEK239vsK562_test', overwrite=True, posttrans=False, pretrnas=False, remap=True, remap_mismatches=0.075, repo_path='/Users/krdav/Google Drive/MCB/Sullivan_lab/tRNA_charging/mim-tRNAseq/mimseq', sampledata='sampleData_HEKvsK562.txt', snp_tolerance=True, species='Hsap', threads=1, trnaout='/Users/krdav/Google Drive/MCB/Sullivan_lab/tRNA_charging/mim-tRNAseq/mimseq/data/hg19-eColitK/hg19_eschColi-tRNAs.out', trnas='/Users/krdav/Google Drive/MCB/Sullivan_lab/tRNA_charging/mim-tRNAseq/mimseq/data/hg19-eColitK/hg19_eColitK.fa')

In [7]:
# This will be wrapped as a function with "args" as input

#####################
# Results directory #
#####################

if not args.outdir.endswith("/"):
    args.outdir = args.outdir + "/"
# Main wrapper
# Integrity check for output folder argument...

try:
    os.mkdir(args.outdir)
except FileExistsError:
    if args.overwrite:
        print('Overwriting: {}'.format(args.outdir))
        shutil.rmtree(args.outdir)
        os.mkdir(args.outdir)
    else:
        raise FileExistsError("Output folder already exists!")
tmp_dir = args.outdir + "/tmp/"
os.mkdir(tmp_dir)
        

###########
# Logging #
###########

now = datetime.datetime.now()
logging.basicConfig(
    format="%(asctime)s [%(levelname)-5.5s] %(message)s",
    level=logging.INFO,
    handlers=[
        logging.FileHandler(args.outdir + "mim-tRNAseq_{}.log".format(now.strftime("%H-%M-%S"))),
        logging.StreamHandler()
    ])
log.info("mim-tRNAseq v{} run with command:".format(version.__version__))
log.info(" ".join(sys.argv))

########
# main #
########

# Load tRNAs and their modifications
trna_obj = TRNA(args)

# Match each sequence in tRNA_dict to value in modomics_dict using BLAST
log.info("\n+------------------------+ \
    \n| Beginning SNP indexing |\
    \n+------------------------+")




map_round = 1 #first round of mapping





2021-05-25 11:16:45,212 [INFO ] mim-tRNAseq v0.3.4.1 run with command:
2021-05-25 11:16:45,213 [INFO ] /Users/krdav/anaconda3/envs/mimseq/lib/python3.7/site-packages/ipykernel_launcher.py -f /Users/krdav/Library/Jupyter/runtime/kernel-33d2f614-513a-403d-a3f0-8585d86fae22.json
2021-05-25 11:16:45,223 [INFO ] 34 introns registered...
2021-05-25 11:16:45,243 [INFO ] Removed 33 introns (including for duplicate tRNAs)
2021-05-25 11:16:45,244 [INFO ] 399 cytosolic and 22 mitochondrial tRNA sequences imported
2021-05-25 11:16:45,247 [INFO ] Downloading modomics database...
2021-05-25 11:16:45,247 [WARNI] Retrieval of Modomics database disabled. Using local files instead...
2021-05-25 11:16:45,252 [INFO ] Parsing Modomics JSON data...
2021-05-25 11:16:45,259 [INFO ] Number of Modomics entries for Escherichia coli: 43
2021-05-25 11:16:45,260 [INFO ] Number of Modomics entries for Homo sapiens: 45


Overwriting: hg19_HEK239vsK562_test/


2021-05-25 11:17:47,833 [INFO ] Cliped extra CCA from 0 sequences.
2021-05-25 11:17:47,859 [INFO ] 
+------------------------+     
| Beginning SNP indexing |    
+------------------------+


In [14]:
class SNPIndex:
    '''Object for SNP index.'''

    def __init__(self, tRNA_dict, modomics_dict, species_set, tmp_dir, args):
        self.tRNA_dict = tRNA_dict
        self.modomics_dict = modomics_dict
        self.species_set = species_set
        self.tmp_dir = tmp_dir
        self.inp_outdir = args.outdir
        self.inp_cluster = args.cluster
        self.inp_cluster_id = args.cluster_id
        self.inp_expnam = args.name

        # Write BED file for alignment
        self.write_bed_file()
        # For use in clustering
        self.anticodon_list = list({tRNA['anticodon'] for tRNA in self.tRNA_dict.values()})
        # Cluster, or not
        if self.inp_cluster:
            self.clust()
        else:
            self.no_clust()

    
    def write_bed_file(self):
        seq_records = defaultdict()
        tRNAbed_fh = open(self.inp_outdir + self.inp_expnam + "_maturetRNA.bed","w")
        for tRNA_id, tRNA in self.tRNA_dict.items():
            # Initialize list of modified sites for each tRNA
            tRNA['modified_idx'] = list()
            tRNA['insosine_idx'] = list()
            with open(self.tmp_dir + tRNA_id + ".fa", "w") as fh_tRNA:
                print('>{}\n{}'.format(tRNA_id, tRNA['seq']), file=fh_tRNA)
            anticodon = re.search('.*tR(NA|X)-.*?-(.*?)-', tRNA_id).group(2)
            tRNA['anticodon'] = anticodon
            # Find matches to modomics where anticodons match
            # and types are the same 
            # (here regex is used to match anticodons with "." in modomics
            # to all possible matching sequences from input tRNAs)
            match = dict()
            for key, mod in self.modomics_dict.items():
                if re.match("^" + mod['anticodon'] + "+$", anticodon) and tRNA['type'] == mod['type']:
                    match[key] = mod
            if len(match) > 0:
                with open(self.tmp_dir + "modomics_match.fasta", "w") as match_fh:
                    for key, mod in match.items():  
                        print('>{}\n{}'.format(key, mod['unmod_seq']), file=match_fh)

                # Call BLAST
                blastn_cline = NcbiblastnCommandline(query=fh_tRNA.name, subject=match_fh.name, task='blastn-short', out=self.tmp_dir + "blast_temp.xml", outfmt=5)
                blastn_cline()

                # Parse XML result and store hit with highest bitscore    
                blast_rec = NCBIXML.read(open(self.tmp_dir + "blast_temp.xml", "r"))
                maxbit = 0
                tophit = ''
                for aln in blast_rec.alignments:
                    for hsp in aln.hsps:
                        if (hsp.bits > maxbit) and (hsp.align_length / aln.length == 1) and (hsp.identities / aln.length == 1):
                            maxbit = hsp.bits
                            tophit = aln.title.split(' ')[0]

                # Return list of all modified positions for the match as long as there is only 1, add to tRNA_dict
                if tophit:
                    tRNA['modified_idx'].append(match[tophit]['modified_idx'])
                    tRNA['insosine_idx'].append(match[tophit]['insosine_idx'])

            # Build seqrecord list for writing
            seq_records[str(tRNA_id)] = SeqRecord(Seq(tRNA['seq'].upper()), id = str(tRNA_id))
            print('{}\t0\t{}\t{}\t1000\t+\n'.format(tRNA_id, str(len(tRNA['seq'])), tRNA_id), file=tRNAbed_fh)
        tRNAbed_fh.close()

        match_count = sum(len(tRNA['modified_idx'])+len(tRNA['insosine_idx']) for tRNA in self.tRNA_dict.values())
        log.info("{} total tRNA gene sequences (undetermined and nmt sequences excluded)".format(len(self.tRNA_dict)))
        log.info("{} sequences with a match to Modomics dataset".format(match_count))

        with open(str(self.inp_outdir + self.inp_expnam + '_tRNATranscripts.fa'), "w") as temptRNATranscripts:
            SeqIO.write(seq_records.values(), temptRNATranscripts, "fasta")

    def no_clust(self):
        pass

    def clust(self):
        pass
    

In [15]:
snpidx_obj = SNPIndex(trna_obj.tRNA_dict, trna_obj.modomics_dict, trna_obj.species_set, tmp_dir, args)

2021-05-25 11:41:48,077 [INFO ] 421 total tRNA gene sequences (undetermined and nmt sequences excluded)
2021-05-25 11:41:48,083 [INFO ] 36 sequences with a match to Modomics dataset


In [25]:
trna_obj.modomics_dict

{'Escherichia_coli_tRNA-Ala-TGC': {'seq': 'GGGGCTATAGCTCAGCDGGGAGAGCGCCTGCTTVGCACGCAGGAG7TCTGCGGTPCGATCCCGCATAGCTCCACCA',
  'type': 'cyto',
  'anticodon': 'TGC',
  'modified_idx': [],
  'unmod_seq': 'GGGGCTATAGCTCAGCTGGGAGAGCGCCTGCTTTGCACGCAGGAGGTCTGCGGTTCGATCCCGCATAGCTCCACCA',
  'insosine_idx': []},
 'Escherichia_coli_tRNA-Ala-GGC': {'seq': 'GGGGCTATAGCTCAGCDGGGAGAGCGCTTGCATGGCATGCAAGAG7TCAGCGGTPCGATCCCGCTTAGCTCCACCA',
  'type': 'cyto',
  'anticodon': 'GGC',
  'modified_idx': [],
  'unmod_seq': 'GGGGCTATAGCTCAGCTGGGAGAGCGCTTGCATGGCATGCAAGAGGTCAGCGGTTCGATCCCGCTTAGCTCCACCA',
  'insosine_idx': []},
 'Escherichia_coli_tRNA-Arg-ACG': {'seq': 'GCATCCG4AGCTCAGCDGGADAGAGTACTCGG%TICG/ACCGAGCG7XCGGAGGTPCGAATCCTCCCGGATGCACCA',
  'type': 'cyto',
  'anticodon': 'ACG',
  'modified_idx': [47],
  'unmod_seq': 'GCATCCGTAGCTCAGCTGGATAGAGTACTCGGCTACGAACCGAGCGGTCGGAGGTTCGAATCCTCCCGGATGCACCA',
  'insosine_idx': [34]},
 'Escherichia_coli_tRNA-Arg-CCG': {'seq': 'GCGCCCGTAGCTCAGCDGGADAGAGCGCTGCC%TCCGKAGGCAGA

In [ ]:
from __future__ import absolute_import
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML
import re, copy, sys, os, shutil, subprocess, logging, glob
from pathlib import Path
from collections import defaultdict
import pandas as pd
import requests
from requests.models import HTTPError
from mimseq.ssAlign import aligntRNA, extraCCA, tRNAclassifier, tRNAclassifier_nogaps, getAnticodon, clusterAnticodon

def modsToSNPIndex(gtRNAdb, tRNAscan_out, mitotRNAs, modifications_table, experiment_name, out_dir, double_cca, snp_tolerance = False, cluster = False, cluster_id = 0.95, posttrans_mod_off = False, pretrnas = False, local_mod = False):
# Builds SNP index needed for GSNAP based on modificaiton data for each tRNA and clusters tRNAs

    nomatch_count = 0
    match_count = 0
    total_snps = 0
    total_inosines = 0
    snp_records = list()
    seq_records = defaultdict()
    anticodon_list = list()
    tRNAbed = open(out_dir + experiment_name + "_maturetRNA.bed","w")

    ''' Contained in TRNA class
    # generate modomics_dict and tRNA_dict
    tRNA_dict, modomics_dict, species = tRNAparser(gtRNAdb, tRNAscan_out, mitotRNAs, modifications_table, posttrans_mod_off, double_cca, pretrnas, local_mod)
    temp_dir = out_dir + "/tmp/"

    try:
        os.mkdir(temp_dir)
    except FileExistsError:
        log.warning("Temp folder present - previous run interrupted? Overwriting old temp files...\n")


    ###################################################
    ## Main code for matching and SNP index building ##
    ###################################################

    # remove spurious CCA seen in some sequences (i.e. genomically encoded or artifact from gtRNAdb or tRNAScan-SE prediction)
    # to do this, remove previously added CCA (intronRemover()) that falls out of canonical structure using ssAlign module

    with open(out_dir + 'tmpRNAseqs.fa', 'w') as fh_out:
        for header, tRNA in tRNA_dict.items():
            print('>{}\n{}'.format(header, tRNA['seq']), file=fh_out)

    stkname = align_tRNA(fh_out.name, outdir, repo_path)
    wrong_seqs = wrong_cca(stkname)
    # Clip the end of the wrong CCA sequences
    for header in wrong_seqs:
        tRNA_dict[header]['seq'] = tRNA_dict[header]['seq'][:-3]

    os.remove(fh_out.name)
    '''

    
    # match each sequence in tRNA_dict to value in modomics_dict using BLAST


    ####################### Put rest into SNPIndex object ########################
    
    
    log.info("\n+------------------------+ \
        \n| Beginning SNP indexing |\
        \n+------------------------+")    


    for seq in tRNA_dict:
        # Initialise list of modified sites for each tRNA
        tRNA_dict[seq]['modified'] = []
        tRNA_dict[seq]['InosinePos'] = []
        temp_tRNAFasta = open(temp_dir + seq + ".fa","w")
        temp_tRNAFasta.write(">" + seq + "\n" + tRNA_dict[seq]['sequence'] + "\n")
        temp_tRNAFasta.close()
        anticodon = re.search('.*tR(NA|X)-.*?-(.*?)-', seq).group(2)
        tRNA_dict[seq]['anticodon'] = anticodon
        if not anticodon in anticodon_list:
            anticodon_list.append(anticodon)
        # find initial possible matches to modomics where anticodons match and types are the same (here regex is used to match anticodons with "." in modomics to all possible matching sequences from input tRNAs)
        match = {k:v for k,v in modomics_dict.items() if re.match("^" + v['anticodon'] + "+$", anticodon) and tRNA_dict[seq]['type'] == v['type']}
        if len(match) >= 1:
            temp_matchFasta = open(temp_dir + "modomicsMatch.fasta","w")
            for i in match:    
                temp_matchFasta.write(">" + i + "\n" + match[i]['unmod_sequence'] + "\n")
            temp_matchFasta.close()

            #blast
            blastn_cline = NcbiblastnCommandline(query = temp_tRNAFasta.name, subject = temp_matchFasta.name, task = 'blastn-short', out = temp_dir + "blast_temp.xml", outfmt = 5)
            blastn_cline()

            #parse XML result and store hit with highest bitscore    
            blast_record = NCBIXML.read(open(temp_dir + "blast_temp.xml","r"))
            maxbit = 0
            tophit = ''
            for alignment in blast_record.alignments:
                for hsp in alignment.hsps:
                    if (hsp.bits > maxbit) and (hsp.align_length / alignment.length == 1) and (hsp.identities / alignment.length == 1):
                        maxbit = hsp.bits
                        tophit = alignment.title.split(' ')[0]
            
            # return list of all modified positions for the match as long as there is only 1, add to tRNA_dict
            if tophit:
                match_count += 1
                tRNA_dict[seq]['modified'] = match[tophit]['modified']
                tRNA_dict[seq]['InosinePos'] = match[tophit]['InosinePos']
            elif len(tophit) == 0:
                nomatch_count += 1
        if len(match) == 0:
            nomatch_count += 1

        # Build seqrecord list for writing
        seq_records[str(seq)] = SeqRecord(Seq(tRNA_dict[seq]['sequence'].upper()), id = str(seq))

        tRNAbed.write(seq + "\t0\t" + str(len(tRNA_dict[seq]['sequence'])) + "\t" + seq + "\t1000\t+\n" )

    tRNAbed.close()

    log.info("{} total tRNA gene sequences (undetermined and nmt sequences excluded)".format(len(tRNA_dict)))
    log.info("{} sequences with a match to Modomics dataset".format(match_count))

    with open(str(out_dir + experiment_name + '_tRNATranscripts.fa'), "w") as temptRNATranscripts:
        SeqIO.write(seq_records.values(), temptRNATranscripts, "fasta")

        
        
        
        
        
        
###################### Finish until here, KD ######################        
        
        
        
        
        
    # if clustering is not activated then write full gff and report on total SNPs written 
    if not cluster:
        coverage_bed = tRNAbed.name
        mod_lists = defaultdict(list)
        Inosine_lists = defaultdict(list)
        with open(out_dir + experiment_name + "_tRNA.gff","w") as tRNAgff, open(out_dir + experiment_name + "isoacceptorInfo.txt","w") as isoacceptorInfo:    
            isoacceptor_dict = defaultdict(int)
            isoacceptorInfo.write("Isoacceptor\tsize\n")
            for seq in tRNA_dict:
                mod_lists[seq] = tRNA_dict[seq]['modified']
                Inosine_lists[seq] = tRNA_dict[seq]['InosinePos']
                tRNAgff.write(seq + "\ttRNAseq\texon\t1\t" + str(len(tRNA_dict[seq]['sequence'])) + "\t.\t+\t0\tgene_id '" + seq + "'\n")
                isoacceptor_group = '-'.join(seq.split("-")[:-2])
                isoacceptor_dict[isoacceptor_group] += 1
            for key, value in isoacceptor_dict.items():
                isoacceptorInfo.write(key + "\t" + str(value) + "\n")
        # generate Stockholm alignment file for all tRNA transcripts and parse additional mods file
        aligntRNA(temptRNATranscripts.name, out_dir)
        additionalMods, additionalInosines = additionalModsParser(species, out_dir)
        # add additional SNPs from extra file to list of modified positions, and ensure non-redundancy with set()
        # index SNPs
        # Format for SNP index (space separated):
        # >snpID chromosomeName:position(1-based) RefMod
        # e.g. >rs111 Homo_sapiens_nmt_tRNA-Leu-TAA-1-1_exp0:29 GN
        for seq in mod_lists:
            additionalMods_sub = {k:v for k, v in additionalMods.items() if k == seq and tRNA_dict[seq]['species'] in v['species']}
            if additionalMods_sub:
                tRNA_dict[seq]['modified'] = list(set(tRNA_dict[seq]['modified'] + additionalMods_sub[seq]['mods']))
                mod_lists[seq] = list(set(mod_lists[seq] + additionalMods_sub[seq]['mods']))

            total_snps += len(mod_lists[seq])

            # Build snp_records as before but with cluster names and non-redundant sets of modifications
            # Position is 1-based for iit_store i.e. pos + 1
            for (index, pos) in enumerate(mod_lists[seq]):
                #if "Gln-TTG" in seq and pos + 1 == 34:
                #    snp_records.append(">" + seq + "_snp" + str(index) + " " + seq + ":" + str(pos + 1) + " " + tRNA_dict[seq]['sequence'][pos].upper() + "C")
                #else:
                snp_records.append(">" + seq + "_snp" + str(index) + " " + seq + ":" + str(pos + 1) + " " + tRNA_dict[seq]['sequence'][pos].upper() + "N")

        for seq in Inosine_lists:
            additionalInosines_sub = {k:v for k, v in additionalInosines.items() if k == seq and tRNA_dict[seq]['species'] in v['species']}
            if additionalInosines_sub:
                tRNA_dict[seq]['InosinePos'] = list(set(tRNA_dict[seq]['InosinePos'] + additionalInosines_sub[seq]['InosinePos']))
                Inosine_lists[seq] = list(set(Inosine_lists[seq] + additionalInosines_sub[seq]['InosinePos']))

            total_inosines += len(Inosine_lists[seq])
            total_snps += len(Inosine_lists[seq])

            for (index, pos) in enumerate(Inosine_lists[seq]):
                # Add inosines to snp index (in addition to changing ref seqence - see below)
                # Ensure only "A" SNPs are tolerated. That is, a G in the reference allows inosines while an A in the snp index allows unmodified reads
                snp_records.append(">" + seq + "_snp" + str(index) + " " + seq + ":" + str(pos + 1) + " " + "GA")


        Inosine_clusters = [cluster for cluster, inosines in Inosine_lists.items() if len(inosines) > 0]

        # edit ref seqs A to G at inosine positions only if snp_tolerance is enabled
        if snp_tolerance:
            for seq in Inosine_lists:
                for pos in Inosine_lists[seq]:
                    seq_records[seq].seq = seq_records[seq].seq[0:pos] + "G" + seq_records[seq].seq[pos+1:]

        with open(str(out_dir + experiment_name + '_tRNATranscripts.fa'), "w") as temptRNATranscripts:
            SeqIO.write(seq_records.values(), temptRNATranscripts, "fasta")

        if total_snps == 0:
            snp_tolerance = False

        log.info("{:,} modifications written to SNP index".format(total_snps))
        log.info("{:,} A to G replacements in reference sequences for inosine modifications".format(total_inosines))
        # empty mismatch dict to avoid error when returning it from this function
        mismatch_dict = defaultdict(list)
        insert_dict = defaultdict(dd_list)
        del_dict = defaultdict(dd_list)
        mod_lists = {tRNA:list() for tRNA in tRNA_dict.keys()}
        Inosine_lists = {tRNA:data['InosinePos'] for tRNA, data in tRNA_dict.items()}
        cluster_perPos_mismatchMembers = defaultdict(dd_list)
        cluster_dict = defaultdict(list)

    ##########################
    # Cluster tRNA sequences #
    ##########################

    elif cluster:

        log.info("**** Clustering tRNA sequences ****")
        log.info("Clustering tRNA sequences by {:.0%} similarity...".format(cluster_id))
        # dictionary of final centroid sequences
        final_centroids = defaultdict()
        # get dictionary of sequences for each anticodon and write to fastas
        for anticodon in anticodon_list:
            seq_set = {k:{'sequence':v['sequence'],'modified':v['modified']} for k,v in tRNA_dict.items() if v['anticodon'] == anticodon}
            with open(temp_dir + anticodon + "_allseqs.fa","w") as anticodon_seqs:
                for sequence in seq_set:
                    anticodon_seqs.write(">" + sequence + "\n" + seq_set[sequence]['sequence'] + "\n")
            # run usearch on each anticodon sequence fatsa to cluster
            cluster_cmd = ["usearch", "-cluster_fast", temp_dir + anticodon + "_allseqs.fa", "-id", str(cluster_id), "-sizeout" ,"-centroids", temp_dir + anticodon + "_centroids.fa", "-uc", temp_dir + anticodon + "_clusters.uc"]
            #cluster_cmd = ["usearch", "-cluster_smallmem", temp_dir + anticodon + "_allseqs.fa", "-id", str(cluster_id), "--sortedby", "other" ,"-sizeout" ,"-centroids", temp_dir + anticodon + "_centroids.fa", "-uc", temp_dir + anticodon + "_clusters.uc"]            #cluster_cmd = "usearch -cluster_fast " + temp_dir + anticodon + "_allseqs.fa -sort length -id " + str(cluster_id) + " -centroids " + temp_dir + anticodon + "_centroids.fa -uc " + temp_dir + anticodon + "_clusters.uc &> /dev/null"
            subprocess.check_call(cluster_cmd, stdout = subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            # sort clusters by size (i.e. number of members in cluster)
            sort_cmd = ["usearch", "-sortbysize", temp_dir + anticodon + "_centroids.fa", "-fastaout", temp_dir + anticodon + "_centroids_sort.fa"]
            subprocess.check_call(sort_cmd, stdout = subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            # recluster based on sorted by size clusters
            final_cluster_cmd = ["usearch", "-cluster_smallmem", temp_dir + anticodon + "_centroids_sort.fa", "-sortedby", "size", "-id", str(cluster_id), "-centroids", temp_dir + anticodon + "_centroidsFinal.fa"]
            subprocess.check_call(final_cluster_cmd, stdout = subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        # combine centroids files into one file
        for filename in glob.glob(temp_dir + "*_centroidsFinal.fa"):
            with open(filename, "r") as fileh:
                with open(temp_dir + "all_centroids.fa", "a") as outh:
                    outh.write(fileh.read())
        centroids = SeqIO.parse(temp_dir + "all_centroids.fa", "fasta")
        for centroid in centroids:
            centroid.id = centroid.id.split(";")[0]
            final_centroids[centroid.id] = SeqRecord(Seq(str(centroid.seq).upper()), id = centroid.id) 

        # read cluster files, get nonredudant set of mod positions of all members of a cluster, create snp_records for writing SNP index
        cluster_pathlist = Path(temp_dir).glob("**/*_clusters.uc")
        mod_lists = defaultdict(list) # stores non-redundant sets of mismatches and mod positions for clusters
        Inosine_lists = defaultdict(list) # stores positions of inosines for clusters
        snp_records = list()
        cluster_dict = defaultdict(list) # info about clusters and members
        mismatch_dict = defaultdict(list) # dictionary of mismatches only (not mod positions - required for exclusion from misincorporation analysis in mmQuant)
        insert_dict = defaultdict(dd_list) # dictionary of insertions in cluster parents - these are not recorded as mismatches but are needed in order to split clusters into isodecoders
        del_dict = defaultdict(dd_list) # dictionary of deletions in cluster parents - these are not recorded as mismatches but are needed in order to split clusters into isodecoders
        cluster_perPos_mismatchMembers = defaultdict(dd_list) # for each cluster, list of members that mismatch at each position corresonding to mismatch dict - used for splitting clusters into isodecoders (splitClusters.py)
        cluster_num = 0
        total_snps = 0
        total_inosines = 0
        clusterbed = open(out_dir + experiment_name + "_clusters.bed","w")
        coverage_bed = clusterbed.name
        clustergff = open(out_dir + experiment_name + "_tRNA.gff","w")
        for path in cluster_pathlist:
            with open(str(path),"r") as cluster_file:
                for line in cluster_file:
                    line = line.strip()

                    # Handle cluster centroids and initialise modified positions list
                    if line.split("\t")[0] == "S":
                        cluster_num += 1
                        cluster_name = line.split("\t")[8].split(";")[0]
                        mod_lists[cluster_name] = tRNA_dict[cluster_name]["modified"]
                        Inosine_lists[cluster_name] = tRNA_dict[cluster_name]['InosinePos']
                        clusterbed.write(cluster_name + "\t0\t" + str(len(tRNA_dict[cluster_name]['sequence'])) + "\t" + cluster_name + "\t1000\t+\n" )
                        clustergff.write(cluster_name + "\ttRNAseq\texon\t1\t" + str(len(tRNA_dict[cluster_name]['sequence'])) + "\t.\t+\t0\tgene_id '" + cluster_name + "'\n")
                        cluster_dict[cluster_name].append(cluster_name)
                
                    # Handle members of clusters
                    elif line.split("\t")[0] == "H":
                        member_name = line.split("\t")[8].split(";")[0]
                        cluster_name = line.split("\t")[9].split(";")[0]
                        compr_aln = line.split("\t")[7]
                        # if member of cluster is 100% identical (i.e. "=" in 8th column of cluster file)
                        if compr_aln == "=":
                            mod_lists[cluster_name] = list(set(mod_lists[cluster_name] + tRNA_dict[member_name]["modified"]))
                            Inosine_lists[cluster_name] = list(set(Inosine_lists[cluster_name] + tRNA_dict[member_name]["InosinePos"]))
                            cluster_dict[cluster_name].append(member_name)
                        
                        # if there are insertions or deletions in the centroid, edit member or centroid sequences to ignore these positions
                        # and edit modified positions list in order to make non-redundant positions list, similar to next else statement
                        elif re.search("[ID]", compr_aln):
                            cluster_seq = tRNA_dict[cluster_name]["sequence"]
                            member_seq = tRNA_dict[member_name]["sequence"]
                            pos = 0
                            adjust_pos_del = 0
                            adjust_pos_ins = 0
                            insertion_pos = list()
                            deletion_pos = list()
                            aln_list = re.split('(.*?[A-Z])', compr_aln)
                            aln_list = list(filter(None, aln_list))

                            for phrase in aln_list:
                                
                                if ("M" in phrase) and (phrase.split("M")[0] != ""):
                                    pos += int(phrase.split("M")[0])
                                elif ("M" in phrase) and (phrase.split("M")[0] == ""):
                                    pos += 1

                                if ("I" in phrase) and (phrase.split("I")[0] != ""):
                                    insert_len = int(phrase.split("I")[0])

                                    for i in range(insert_len):
                                        insertion_pos.append(pos+i)
                                        #pos += 1
                                elif ("I" in phrase) and (phrase.split("I")[0] == ""):
                                    insertion_pos.append(pos)
                                    #pos += 1

                                if ("D" in phrase) and (phrase.split("D")[0] != ""):
                                    delete_len = int(phrase.split("D")[0])

                                    for i in range(delete_len):
                                        deletion_pos.append(pos)
                                        pos += 1
                                elif ("D" in phrase) and (phrase.split("D")[0] == ""):
                                    deletion_pos.append(pos)
                                    pos += 1

                            for delete in deletion_pos:
                                adjust_pos_len = 0
                                for insert in insertion_pos:
                                    if insert < delete:
                                        adjust_pos_len -= 1
                                new_delete = delete + adjust_pos_del + adjust_pos_len
                                member_seq = member_seq[ :new_delete] + member_seq[new_delete+1: ]
                                del_dict[cluster_name][new_delete].append(member_name)
                                adjust_pos_del -= 1

                            for index, insert in enumerate(insertion_pos):
                                adjust_pos_len = 0
                                for delete in deletion_pos:
                                    if delete < insert:
                                        adjust_pos_len -= 1
                                #if index != 0:
                                #    if insert == insertion_pos[index-1] + 1:
                                #        adjust_pos_ins -= 1
                                new_insert = insert + adjust_pos_len + adjust_pos_ins
                                member_seq = member_seq[ :new_insert] + cluster_seq[insert] + member_seq[new_insert: ]
                                #insert_dict[cluster_name][new_insert-1].append(member_name)
                                insert_dict[cluster_name][new_insert].append(member_name)
                                #cluster_seq = cluster_seq[ :new_insert] + cluster_seq[new_insert+1: ]

                            mismatches = [i for i in range(len(member_seq)) if member_seq[i].upper() != cluster_seq[i].upper()]
                            mismatch_dict[cluster_name] = list(set(mismatch_dict[cluster_name] + mismatches))
                            for mismatch in mismatches:
                                cluster_perPos_mismatchMembers[cluster_name][mismatch].append(member_name)
                            member_mods = list(set(tRNA_dict[member_name]["modified"] + mismatches))
                            member_Inosines = tRNA_dict[member_name]["InosinePos"]
                            mod_lists[cluster_name] = list(set(mod_lists[cluster_name] + member_mods))
                            Inosine_lists[cluster_name] = list(set(Inosine_lists[cluster_name] + member_Inosines))
                            cluster_dict[cluster_name].append(member_name)

                        # handle members that are not exact sequence matches but have no indels either
                        # find mismatches and build non-redundant set
                        else:
                            cluster_seq = tRNA_dict[cluster_name]["sequence"]
                            member_seq = tRNA_dict[member_name]["sequence"]
                            mismatches = [i for i in range(len(member_seq)) if member_seq[i].upper() != cluster_seq[i].upper()]
                            mismatch_dict[cluster_name] = list(set(mismatch_dict[cluster_name] + mismatches))
                            for mismatch in mismatches:
                                cluster_perPos_mismatchMembers[cluster_name][mismatch].append(member_name)
                            member_mods = list(set(tRNA_dict[member_name]["modified"] + mismatches))
                            member_Inosines = tRNA_dict[member_name]["InosinePos"]
                            mod_lists[cluster_name] = list(set(mod_lists[cluster_name] + member_mods))
                            Inosine_lists[cluster_name] = list(set(Inosine_lists[cluster_name] + member_Inosines))
                            cluster_dict[cluster_name].append(member_name)

        clusterbed.close()

        # Write cluster information to tsv and get number of unique sequences per cluster (i.e. isodecoders) for read count splitting
        with open(out_dir + experiment_name + "clusterInfo.txt","w") as clusterInfo, open(out_dir + experiment_name + "isoacceptorInfo.txt","w") as isoacceptorInfo:
            isoacceptor_dict = defaultdict(int)
            clusterInfo.write("tRNA\tcluster_num\tcluster_size\tparent\n")
            isoacceptorInfo.write("Isoacceptor\tsize\n")
            for key, value in cluster_dict.items():
                cluster_num = list(cluster_dict.keys()).index(key)
                for member in value:
                    clusterInfo.write("{}\t{}\t{}\t{}\n".format(member, cluster_num, len(value), key))
                    isoacceptor_group = '-'.join(member.split("-")[:-2])
                    isoacceptor_dict[isoacceptor_group] += 1
            for key, value in isoacceptor_dict.items():
                isoacceptorInfo.write(key + "\t" + str(value) + "\n")
        with open(str(out_dir + experiment_name + '_clusterTranscripts.fa'), "w") as clusterTranscripts:
            SeqIO.write(final_centroids.values(), clusterTranscripts, "fasta")

        # generate Stockholm alignment file for cluster transcripts and process additional mods file
        aligntRNA(clusterTranscripts.name, out_dir)
        additionalMods, additionalInosines = additionalModsParser(species, out_dir)

        log.info("{} clusters created from {} tRNA sequences".format(len(cluster_dict),len(tRNA_dict)))

        # update mod_lists with additional mods and write SNP index
        for cluster in mod_lists:
            additionalMods_sub = {k:v for k, v in additionalMods.items() if k == cluster and tRNA_dict[cluster]['species'] in v['species']}
            if additionalMods_sub:
                tRNA_dict[cluster]['modified'] = list(set(tRNA_dict[cluster]['modified'] + additionalMods_sub[cluster]['mods']))
                mod_lists[cluster] = list(set(mod_lists[cluster] + additionalMods_sub[cluster]['mods']))

            total_snps += len(mod_lists[cluster])

            for (index, pos) in enumerate(mod_lists[cluster]):
                # Build snp_records as before but with cluster names and non-redundant sets of modifications
                # Position is 1-based for iit_store i.e. pos + 1
                #if "Gln-TTG" in cluster and pos + 1 == 34:
                #    snp_records.append(">" + cluster + "_snp" + str(index) + " " + cluster + ":" + str(pos + 1) + " " + tRNA_dict[cluster]['sequence'][pos].upper() + "C")
                #else:
                snp_records.append(">" + cluster + "_snp" + str(index) + " " + cluster + ":" + str(pos + 1) + " " + tRNA_dict[cluster]['sequence'][pos].upper() + "N")

        for cluster in Inosine_lists:
            additionalInosines_sub = {k:v for k, v in additionalInosines.items() if k == cluster and tRNA_dict[cluster]['species'] in v['species']}
            if additionalInosines_sub:
                tRNA_dict[cluster]['InosinePos'] = list(set(tRNA_dict[cluster]['InosinePos'] + additionalInosines_sub[cluster]['InosinePos']))
                Inosine_lists[cluster] = list(set(Inosine_lists[cluster] + additionalInosines_sub[cluster]['InosinePos']))

            total_inosines += len(Inosine_lists[cluster])
            total_snps += len(Inosine_lists[cluster])

            for (index, pos) in enumerate(Inosine_lists[cluster]):
                # Add inosines to snp index (in addition to changing ref seqence - see below)
                # Ensure only "A" SNPs are tolerated. That is, a G in the reference allows inosines while an A in the snp index allows unmodified reads
                snp_records.append(">" + cluster + "_snp" + str(index) + " " + cluster + ":" + str(pos + 1) + " " + "GA")


        # edit ref seqs A to G at inosine positions if snp_tolerance is enabled
        if snp_tolerance:
            for cluster in Inosine_lists:
                for pos in Inosine_lists[cluster]:
                    final_centroids[cluster].seq = final_centroids[cluster].seq[0:pos] + "G" + final_centroids[cluster].seq[pos+1:]

        Inosine_clusters = [cluster for cluster, inosines in Inosine_lists.items() if len(inosines) > 0]

        # rewrite edited cluster transcripts
        with open(str(out_dir + experiment_name + '_clusterTranscripts.fa'), "w") as clusterTranscripts:
            SeqIO.write(final_centroids.values(), clusterTranscripts, "fasta")

        if total_snps == 0:
            snp_tolerance = False        

        log.info("{:,} modifications written to SNP index".format(total_snps))
        log.info("{:,} A to G replacements in reference sequences for inosine modifications".format(total_inosines))        
    
    # write outputs for indexing 
    with open(out_dir + experiment_name + "_modificationSNPs.txt", "w") as snp_file:
        for item in snp_records:
            snp_file.write('{}\n'.format(item))
    
    shutil.rmtree(temp_dir)
    
    # Return coverage_bed (either tRNAbed or clusterbed depending on --cluster) for coverage calculation method
    return(coverage_bed, snp_tolerance, mismatch_dict, insert_dict, del_dict, mod_lists, Inosine_lists, Inosine_clusters, tRNA_dict, cluster_dict, cluster_perPos_mismatchMembers)

In [ ]:
coverage_bed, snp_tolerance, mismatch_dict, insert_dict, del_dict, mod_lists, Inosine_lists, Inosine_clusters, tRNA_dict, cluster_dict, cluster_perPos_mismatchMembers = modsToSNPIndex(trnas, trnaout, mito_trnas, modifications, name, out, double_cca, snp_tolerance, cluster, cluster_id, posttrans, pretrnas, local_mod)

In [29]:
args

Namespace(cca=True, cluster=True, cluster_id=0.95, control_cond='HEK293T', cov_diff=0.5, double_cca=False, keep_temp=False, local_mod=False, max_multi=1, min_cov=2000.0, misinc_thresh=0.1, mismatches=0.1, mito='/Users/krdav/Google Drive/MCB/Sullivan_lab/tRNA_charging/mim-tRNAseq/data/hg19-eColitK/hg19-mitotRNAs.fa', name='hg19_test', out='hg19_HEK239vsK562', posttrans=False, pretrnas=False, remap=True, remap_mismatches=0.075, sampledata='sampleData_HEKvsK562.txt', snp_tolerance=True, species='Hsap', threads=1, trnaout='/Users/krdav/Google Drive/MCB/Sullivan_lab/tRNA_charging/mim-tRNAseq/data/hg19-eColitK/hg19_eschColi-tRNAs.out', trnas='/Users/krdav/Google Drive/MCB/Sullivan_lab/tRNA_charging/mim-tRNAseq/data/hg19-eColitK/hg19_eColitK.fa')